In [ ]:
# Import everything
from distribution_monad import Prob, uniform, weighted, bernoulli
from powerset_monad import Powerset, singleton, empty, choice, maybe
from giry_monad import GiryMeasure, dirac, discrete_uniform, normal, uniform as giry_uniform, exponential
from identity_monad import Identity, logical_and, logical_or, logical_not, true, false, Proposition, truth_table, verify_law
from probabilistic_semantics import (ProgramState, ProbabilisticFormula, Atom, 
                                     var_equals, var_greater, var_less, probabilistic_assignment, 
                                     forall, exists, truth as prob_truth, falsity, ProbabilisticLogicGiry)
from ltn_semantics import (ltn_forall, ltn_exists, ltn_assignment, 
                          ltn_forall_strict, ltn_forall_mean, 
                          ltn_exists_strict, ltn_exists_mean)

In [50]:
# Example 1: Fair coin
coin = uniform(['H', 'T'])
print("Fair coin:", coin)

# Example 2: Die
die = uniform(list(range(1,7)))
print("Die:", die)

# Example 3: Two coin flips using monadic bind
two_flips = coin.bind(lambda x1: 
            coin.bind(lambda x2: 
            Prob.unit((x1, x2))))
print("Two coin flips:", two_flips)

# Example 4: Sum of two dice using bind
two_dice_sum = die.bind(lambda d1:
               die.bind(lambda d2:
               Prob.unit(d1 + d2)))
print("Sum of two dice:", two_dice_sum)

# Example 5: Using map
squared_die = die.map(lambda x: x ** 2)
print("Squared die values:", squared_die)

# Example 6: Conditional probability - at least one head in two flips
at_least_one_head = two_flips.filter(lambda pair: 'H' in pair)
print("At least one head in two flips:", at_least_one_head)

# Example 7: Expected value
print("Expected value of die:", die.expected_value())

# Example 8: Sampling
print("5 samples from die:", [die.sample() for _ in range(5)])


Fair coin: Prob({'H': 0.5, 'T': 0.5})
Die: Prob({1: 0.16666666666666666, 2: 0.16666666666666666, 3: 0.16666666666666666, 4: 0.16666666666666666, 5: 0.16666666666666666, 6: 0.16666666666666666})
Two coin flips: Prob({('H', 'H'): 0.25, ('H', 'T'): 0.25, ('T', 'H'): 0.25, ('T', 'T'): 0.25})
Sum of two dice: Prob({7: 0.16666666666666669, 6: 0.1388888888888889, 8: 0.1388888888888889, 5: 0.1111111111111111, 9: 0.1111111111111111, 4: 0.08333333333333333, 10: 0.08333333333333333, 3: 0.05555555555555555, 11: 0.05555555555555555, 2: 0.027777777777777776, 12: 0.027777777777777776})
Squared die values: Prob({1: 0.16666666666666666, 4: 0.16666666666666666, 9: 0.16666666666666666, 16: 0.16666666666666666, 25: 0.16666666666666666, 36: 0.16666666666666666})
At least one head in two flips: Prob({('H', 'H'): 0.3333333333333333, ('H', 'T'): 0.3333333333333333, ('T', 'H'): 0.3333333333333333})
Expected value of die: 3.5
5 samples from die: [4, 4, 2, 5, 1]


In [51]:
# Example 9: Biased Die
Bdie = weighted([(1, 1/8), (2, 1/8), (3, 1/8), (4, 1/8), (5, 1/8), (6, 3/8)])
print("Biased Die:", Bdie)

# Example 10: Sum of two biased dice using bind
two_dice_sum = Bdie.bind(lambda d1:
               Bdie.bind(lambda d2:
               Prob.unit(d1 + d2)))
print("Sum of two biased dice:", two_dice_sum)

# Example 11: Squared biased die using map
squared_die = Bdie.map(lambda x: x ** 2)
print("Squared biased die values:", squared_die)

# Example 12: Expected value of biased die
print("Expected value of biased die:", Bdie.expected_value())

# Example 13: Sampling from biased die
print("5 samples from biased die:", [Bdie.sample() for _ in range(5)])

# Example 14: Test whether sum is even
even_sum = die.bind(lambda d1:
            die.bind(lambda d2:
            Prob.unit((d1 + d2) % 2 == 0)))
print("Sum of two dice is even:", even_sum)

# Example 15: Test whether sum equals 12
twelve_sum = die.bind(lambda d1:
            die.bind(lambda d2:
            Prob.unit((d1 + d2)  == 12)))
print("Sum of two dice is equal to 12:", twelve_sum)  

Biased Die: Prob({6: 0.375, 1: 0.125, 2: 0.125, 3: 0.125, 4: 0.125, 5: 0.125})
Sum of two biased dice: Prob({7: 0.15625, 8: 0.140625, 12: 0.140625, 9: 0.125, 10: 0.109375, 11: 0.09375, 6: 0.078125, 5: 0.0625, 4: 0.046875, 3: 0.03125, 2: 0.015625})
Squared biased die values: Prob({36: 0.375, 1: 0.125, 4: 0.125, 9: 0.125, 16: 0.125, 25: 0.125})
Expected value of biased die: 4.125
5 samples from biased die: [2, 6, 3, 2, 3]
Sum of two dice is even: Prob({True: 0.5, False: 0.5})
Sum of two dice is equal to 12: Prob({False: 0.9722222222222222, True: 0.027777777777777783})


In [52]:
# Powerset Monad Examples - Non-deterministic Computation

# Example 16: Simple non-deterministic choice
coin_flip = choice('H', 'T')
print("Coin flip:", coin_flip)

# Example 17: Deterministic computation
certain = singleton(42)
print("Certain value:", certain)

# Example 18: Non-deterministic computation using bind
# Choose a number, then add 1 or 2 to it
numbers = choice(1, 2, 3)
incremented = numbers.bind(lambda x: choice(x + 1, x + 2))
print("Numbers incremented by 1 or 2:", incremented)

# Example 19: Chaining non-deterministic computations
# Start with a choice, square it, then add 1 or subtract 1
computation = (choice(2, 3)
              .bind(lambda x: singleton(x * x))
              .bind(lambda x: choice(x + 1, x - 1)))
print("Complex computation:", computation)

# Example 20: Using map for deterministic transformation
doubled = choice(1, 2, 3).map(lambda x: x * 2)
print("Doubled values:", doubled)

# Example 21: Filtering non-deterministic results
large_numbers = choice(1, 5, 10, 15).filter(lambda x: x > 7)
print("Large numbers only:", large_numbers)

# Example 22: Combining powersets
set1 = choice('a', 'b')
set2 = choice('c', 'd')
combined = set1.bind(lambda x: set2.bind(lambda y: singleton((x, y))))
print("Cartesian product:", combined)

# Example 23: Maybe computation (success or failure)
maybe_value = maybe(100)
safe_division = maybe_value.bind(lambda x: singleton(x / 2) if x is not None else empty())
print("Maybe computation:", safe_division)

# Example 24: Sampling from non-deterministic computation
random_samples = [choice(1, 2, 3, 4, 5).sample() for _ in range(10)]
print("Random samples:", random_samples)

# Example 25: Set operations
set_a = choice(1, 2, 3)
set_b = choice(2, 3, 4)
print("Union:", set_a.union(set_b))
print("Intersection:", set_a.intersection(set_b))
print("Difference:", set_a.difference(set_b)) 

Coin flip: Powerset({'T', 'H'})
Certain value: Powerset({42})
Numbers incremented by 1 or 2: Powerset({2, 3, 4, 5})
Complex computation: Powerset({8, 10, 3, 5})
Doubled values: Powerset({2, 4, 6})
Large numbers only: Powerset({10, 15})
Cartesian product: Powerset({('a', 'c'), ('a', 'd'), ('b', 'c'), ('b', 'd')})
Maybe computation: Powerset({50.0})
Random samples: [3, 5, 1, 5, 3, 5, 2, 4, 3, 2]
Union: Powerset({1, 2, 3, 4})
Intersection: Powerset({2, 3})
Difference: Powerset({1})


In [53]:
# Giry Monad Examples - Probability Measures on Measurable Spaces

# Example 26: Dirac delta measure
delta_5 = dirac(5)
print("Dirac δ_5:", delta_5)
print("δ_5({5}):", delta_5.measure(lambda x: x == 5))
print("δ_5({1,2,3}):", delta_5.measure(lambda x: x in [1,2,3]))

# Example 27: Discrete uniform distribution
dice = discrete_uniform([1, 2, 3, 4, 5, 6])
print("\nDice:", dice)
print("P(X ≤ 3):", dice.measure(lambda x: x <= 3))
print("E[X]:", dice.expected_value())

# Example 28: Normal distribution
gaussian = normal(0, 1)
print("\nStandard normal:", gaussian)
print("E[X]:", gaussian.expected_value())
print("E[X²]:", gaussian.expected_value(lambda x: x**2))

# Example 29: Monadic bind with Dirac measures
# Start with δ_5, then map x → δ_{x+1}
result = delta_5.bind(lambda x: dirac(x + 1))
print("\nδ_5 >>= (x → δ_{x+1}):", result)
print("Sample:", result.sample())

# Example 30: Bind with discrete distribution
# Roll dice, then flip coins based on the result
dice_then_coins = dice.bind(lambda x: discrete_uniform([0, 1] * x))
print("\nDice then coins:", dice_then_coins)
print("Sample:", dice_then_coins.sample())

# Example 31: Continuous distribution bind
# Sample from normal, then create exponential with that rate
normal_to_exp = normal(2, 0.5).bind(lambda x: exponential(abs(x)) if x > 0 else dirac(0))
print("\nNormal to exponential:", normal_to_exp)
print("Sample:", normal_to_exp.sample())

# Example 32: Pushforward measure (map)
squared_dice = dice.map(lambda x: x**2)
print("\nSquared dice:", squared_dice)
print("E[X²]:", squared_dice.expected_value())

# Example 33: Measure of intervals for continuous distributions
unit_uniform = giry_uniform(0, 1)
print("\nUnit uniform:", unit_uniform)
print("P(X ∈ [0.3, 0.7]):", unit_uniform.measure(lambda x: 0.3 <= x <= 0.7))

# Example 34: Composition of measures
# Sample from uniform, then normal with that mean
composition = unit_uniform.bind(lambda mu: normal(mu, 0.1))
print("\nComposition:", composition)
print("Sample:", composition.sample())


Dirac δ_5: δ_5
δ_5({5}): 1.0
δ_5({1,2,3}): 0.0

Dice: GiryMeasure(discrete_uniform, discrete)
P(X ≤ 3): 0.5
E[X]: 3.5

Standard normal: GiryMeasure(N(0,1), continuous)
E[X]: 0.004220828490528678
E[X²]: 1.0045216518947682

δ_5 >>= (x → δ_{x+1}): GiryMeasure(bind(δ_5), discrete)
Sample: 6

Dice then coins: GiryMeasure(bind(discrete_uniform), discrete)
Sample: 1

Normal to exponential: GiryMeasure(bind(N(2,0.5)), discrete)
Sample: 0.42555684144693284

Squared dice: GiryMeasure(map(discrete_uniform), discrete)
E[X²]: 15.166666666666666

Unit uniform: GiryMeasure(U(0,1), continuous)
P(X ∈ [0.3, 0.7]): 0.3999

Composition: GiryMeasure(bind(U(0,1)), discrete)
Sample: 0.14612966979914604


In [54]:
# Identity Monad Examples - Boolean Logic

# Example 35: Basic boolean operations
print("=== Identity Monad - Basic Boolean Operations ===")
t = true()
f = false()

print(f"True: {t}")
print(f"False: {f}")
print(f"NOT True: {t.bind(lambda x: logical_not(x))}")
print(f"NOT False: {f.bind(lambda x: logical_not(x))}")

# Example 36: Combining operations with bind
print("\n=== Combining Operations ===")
result1 = t.bind(lambda x: f.bind(lambda y: logical_and(x, y)))
result2 = t.bind(lambda x: f.bind(lambda y: logical_or(x, y)))
print(f"True AND False: {result1}")
print(f"True OR False: {result2}")

# Example 37: Using map for transformations
print("\n=== Using Map ===")
negated = t.map(lambda x: not x)
print(f"map(NOT, True): {negated}")

# Example 38: Chaining operations
print("\n=== Chaining Operations ===")
# (True AND False) OR (NOT False)
complex_expr = (t.bind(lambda x: 
                f.bind(lambda y: 
                logical_and(x, y)))
               .bind(lambda result:
                f.bind(lambda z:
                logical_or(result, not z))))
print(f"(True AND False) OR (NOT False): {complex_expr}")

# Example 39: Propositional logic evaluation
print("\n=== Propositional Logic ===")
prop1 = Proposition("p AND q")
prop2 = Proposition("NOT (NOT p OR NOT q)")  # De Morgan's law

assignments = {'p': True, 'q': False}
result1 = prop1.evaluate(assignments)
result2 = prop2.evaluate(assignments)

print(f"p AND q with p=True, q=False: {result1}")
print(f"NOT (NOT p OR NOT q) with p=True, q=False: {result2}")

# Example 40: Truth tables
print("\n=== Truth Tables ===")
prop = Proposition("p AND q")
table = truth_table(prop, ['p', 'q'])

print("Truth table for 'p AND q':")
print("p\tq\tresult")
print("-" * 15)
for row in table:
    print(f"{row[0]}\t{row[1]}\t{row[2]}")

# Example 41: Verifying Boolean algebra laws
print("\n=== Boolean Algebra Laws ===")

# De Morgan's Laws
law1_left = Proposition("NOT (p AND q)")
law1_right = Proposition("(NOT p) OR (NOT q)")
verify_law("De Morgan's Law 1", law1_left, law1_right, ['p', 'q'])

law2_left = Proposition("NOT (p OR q)")
law2_right = Proposition("(NOT p) AND (NOT q)")
verify_law("De Morgan's Law 2", law2_left, law2_right, ['p', 'q'])

# Distributive Laws
dist1_left = Proposition("p AND (q OR r)")
dist1_right = Proposition("(p AND q) OR (p AND r)")
verify_law("Distributive Law 1", dist1_left, dist1_right, ['p', 'q', 'r'])

# Absorption Laws
abs1_left = Proposition("p AND (p OR q)")
abs1_right = Proposition("p")
verify_law("Absorption Law 1", abs1_left, abs1_right, ['p', 'q'])

# Example 42: Monadic composition laws
print("\n=== Monad Laws Verification ===")

# Left identity: unit(a) >>= f ≡ f(a)
a = True
f = lambda x: logical_not(x)
left_id_lhs = Identity.unit(a).bind(f)
left_id_rhs = f(a)
print(f"Left identity: {left_id_lhs} ≡ {left_id_rhs} -> {left_id_lhs.get() == left_id_rhs.get()}")

# Right identity: m >>= unit ≡ m
m = Identity(True)
right_id_lhs = m.bind(Identity.unit)
right_id_rhs = m
print(f"Right identity: {right_id_lhs} ≡ {right_id_rhs} -> {right_id_lhs == right_id_rhs}")

# Associativity: (m >>= f) >>= g ≡ m >>= (λx -> f(x) >>= g)
g = lambda x: Identity(x and True)
assoc_lhs = m.bind(f).bind(g)
assoc_rhs = m.bind(lambda x: f(x).bind(g))
print(f"Associativity: {assoc_lhs} ≡ {assoc_rhs} -> {assoc_lhs == assoc_rhs}")


=== Identity Monad - Basic Boolean Operations ===
True: Identity(True)
False: Identity(False)
NOT True: Identity(False)
NOT False: Identity(True)

=== Combining Operations ===
True AND False: Identity(False)
True OR False: Identity(True)

=== Using Map ===
map(NOT, True): Identity(False)

=== Chaining Operations ===
(True AND False) OR (NOT False): Identity(True)

=== Propositional Logic ===
p AND q with p=True, q=False: Identity(False)
NOT (NOT p OR NOT q) with p=True, q=False: Identity(False)

=== Truth Tables ===
Truth table for 'p AND q':
p	q	result
---------------
False	False	False
True	False	False
False	True	False
True	True	True

=== Boolean Algebra Laws ===
✓ De Morgan's Law 1: VERIFIED
✓ De Morgan's Law 2: VERIFIED
✓ Distributive Law 1: VERIFIED
✓ Absorption Law 1: VERIFIED

=== Monad Laws Verification ===
Left identity: Identity(False) ≡ Identity(False) -> True
Right identity: Identity(True) ≡ Identity(True) -> True
Associativity: Identity(False) ≡ Identity(False) -> True


In [55]:
# Probabilistic Logic with Giry Monad - BL-Algebra Semantics

# Example 43: Basic probabilistic logic operations  
print("=== Probabilistic Logic with BL-Algebra Semantics ===")

# Create initial state
initial_state = ProgramState({"x": 5, "y": 3})

# Create atomic formulas
x_positive = var_greater("x", 0)
y_positive = var_greater("y", 0)
x_large = var_greater("x", 10)

print(f"x > 0: {x_positive.evaluate(initial_state)}")
print(f"y > 0: {y_positive.evaluate(initial_state)}")
print(f"x > 10: {x_large.evaluate(initial_state)}")

# Test BL-Algebra logical operations
conjunction = x_positive & y_positive  # F ∧ G := [[F]] · [[G]]
disjunction = x_positive | x_large     # F ∨ G := [[F]] + [[G]] - [[F]] · [[G]]
negation = ~x_large                    # ¬F := 1 - [[F]]
implication = x_large.implies(y_positive)  # F → G := 1 - [[F]] + [[F]] · [[G]]

print(f"(x > 0) ∧ (y > 0): {conjunction.evaluate(initial_state)}")
print(f"(x > 0) ∨ (x > 10): {disjunction.evaluate(initial_state)}")
print(f"¬(x > 10): {negation.evaluate(initial_state)}")
print(f"(x > 10) → (y > 0): {implication.evaluate(initial_state)}")

# Example 44: Probabilistic assignment with integration
print("\n=== Probabilistic Assignment ===")

# ⟦x := m(T)(F)⟧ := ∫_{a∈I(s_m)} ⟦F⟧_ν[x↦a] dρ_m(· | T)(a)
normal_measure = normal(0, 1)
x_positive_after = var_greater("x", 0)

assignment = probabilistic_assignment("x", normal_measure, x_positive_after)
prob_x_positive = assignment.evaluate(initial_state)

print(f"P(x > 0) after x := Normal(0,1): {prob_x_positive:.3f}")

# Example 45: Conditional assignment
print("\n=== Conditional Assignment ===")

# x := Normal(0, 1) | x > -1, evaluate P(x > 0)
condition = var_greater("x", -1)
conditional_assignment = probabilistic_assignment("x", normal_measure, x_positive_after, condition)

prob_x_positive_conditional = conditional_assignment.evaluate(initial_state)
print(f"P(x > 0) after x := Normal(0,1) | x > -1: {prob_x_positive_conditional:.3f}")

# Example 46: Quantification as inf/sup
print("\n=== Quantification ===")

# ⟦∀x:s F⟧ := inf_{a∈I(s)} ⟦F⟧_ν[x↦a]
domain = [1, 2, 3]
universal_formula = forall("x", domain, var_greater("x", 0))
print(f"∀x ∈ {{1,2,3}}: x > 0 = {universal_formula.evaluate(initial_state)}")

# ⟦∃x:s F⟧ := sup_{a∈I(s)} ⟦F⟧_ν[x↦a]
domain2 = [-1, 0, 1]
existential_formula = exists("x", domain2, var_greater("x", 0))
print(f"∃x ∈ {{-1,0,1}}: x > 0 = {existential_formula.evaluate(initial_state)}")

# Example 47: Complex probabilistic reasoning
print("\n=== Complex Probabilistic Reasoning ===")

# Probabilistic program: x := U(0,1), y := N(x,0.1), P((x > 0.5) ∧ (y > 0.6))
uniform_measure = giry_uniform(0, 1)

def complex_reasoning():
    total_prob = 0.0
    n_samples = 1000
    
    for _ in range(n_samples):
        x_val = uniform_measure.sample()
        y_measure = normal(x_val, 0.1)
        y_val = y_measure.sample()
        
        # Create state with both assignments
        state = ProgramState({"x": x_val, "y": y_val})
        
        # Evaluate (x > 0.5) ∧ (y > 0.6)
        formula = var_greater("x", 0.5) & var_greater("y", 0.6)
        total_prob += formula.evaluate(state)
    
    return total_prob / n_samples

complex_prob = complex_reasoning()
print(f"P((x > 0.5) ∧ (y > 0.6)) after x := U(0,1), y := N(x,0.1): {complex_prob:.3f}")

# Example 48: Using Giry monad for state distributions
print("\n=== Giry Monad State Distributions ===")

# Create distribution over program states
states = [
    ProgramState({"x": 1, "y": 2}),
    ProgramState({"x": 3, "y": 4}),
    ProgramState({"x": 5, "y": 6})
]

state_measure = ProbabilisticLogicGiry(
    distribution_type="discrete",
    values=states,
    probabilities=[0.2, 0.3, 0.5],
    name="state_distribution"
)

# Evaluate formula over state distribution
formula = var_greater("x", 2) & var_greater("y", 3)
result_measure = state_measure.evaluate_formula(formula)

print(f"P((x > 2) ∧ (y > 3)) over state distribution: {result_measure.sample():.3f}")


=== Probabilistic Logic with BL-Algebra Semantics ===
x > 0: 1.0
y > 0: 1.0
x > 10: 0.0
(x > 0) ∧ (y > 0): 1.0
(x > 0) ∨ (x > 10): 1.0
¬(x > 10): 1.0
(x > 10) → (y > 0): 1.0

=== Probabilistic Assignment ===
P(x > 0) after x := Normal(0,1): 0.514

=== Conditional Assignment ===
P(x > 0) after x := Normal(0,1) | x > -1: 0.525

=== Quantification ===
∀x ∈ {1,2,3}: x > 0 = 1.0
∃x ∈ {-1,0,1}: x > 0 = 1.0

=== Complex Probabilistic Reasoning ===
P((x > 0.5) ∧ (y > 0.6)) after x := U(0,1), y := N(x,0.1): 0.396

=== Giry Monad State Distributions ===
P((x > 2) ∧ (y > 3)) over state distribution: 0.800


In [56]:
# BL-Algebra Verification with Diverse Probability Values
print("\n=== BL-Algebra Properties Verification ===")

# Create probabilistic formulas that evaluate to various probability values
# Using probabilistic assignments to get intermediate probability values

# Test Case 1: Formulas with probabilities ~0.3 and ~0.7
print("\n--- Test Case 1: P(F) ≈ 0.3, P(G) ≈ 0.7 ---")

# Create formulas that will give us intermediate probabilities
state1 = ProgramState({})

# F: Sample from U(0,1) and check if > 0.7 (gives P ≈ 0.3)
uniform_01 = giry_uniform(0, 1)
f_prob = probabilistic_assignment("x", uniform_01, var_greater("x", 0.7))

# G: Sample from U(0,1) and check if > 0.3 (gives P ≈ 0.7)  
g_prob = probabilistic_assignment("y", uniform_01, var_greater("y", 0.3))

f_val = f_prob.evaluate(state1)
g_val = g_prob.evaluate(state1)

print(f"P(F) = {f_val:.3f}")
print(f"P(G) = {g_val:.3f}")

# Test BL-algebra operations
conjunction_prob = f_val * g_val
disjunction_prob = f_val + g_val - f_val * g_val  
negation_f_prob = 1.0 - f_val
implication_prob = 1.0 - f_val + f_val * g_val

print(f"P(F ∧ G) = P(F) × P(G) = {conjunction_prob:.3f}")
print(f"P(F ∨ G) = P(F) + P(G) - P(F)×P(G) = {disjunction_prob:.3f}")
print(f"P(¬F) = 1 - P(F) = {negation_f_prob:.3f}")
print(f"P(F → G) = 1 - P(F) + P(F)×P(G) = {implication_prob:.3f}")

# Test Case 2: Using normal distributions for smoother probabilities
print("\n--- Test Case 2: Normal Distribution Probabilities ---")

state2 = ProgramState({})

# F: Sample from N(0,1) and check if > 0 (gives P ≈ 0.5)
normal_std = normal(0, 1)
f_normal = probabilistic_assignment("x", normal_std, var_greater("x", 0))

# G: Sample from N(0.5,1) and check if > 0 (gives P ≈ 0.69)
normal_shifted = normal(0.5, 1)
g_normal = probabilistic_assignment("y", normal_shifted, var_greater("y", 0))

f_norm_val = f_normal.evaluate(state2)
g_norm_val = g_normal.evaluate(state2)

print(f"P(F) = P(N(0,1) > 0) = {f_norm_val:.3f}")
print(f"P(G) = P(N(0.5,1) > 0) = {g_norm_val:.3f}")

# Test BL-algebra operations
conjunction_norm = f_norm_val * g_norm_val
disjunction_norm = f_norm_val + g_norm_val - f_norm_val * g_norm_val
negation_norm = 1.0 - f_norm_val
implication_norm = 1.0 - f_norm_val + f_norm_val * g_norm_val

print(f"P(F ∧ G) = {conjunction_norm:.3f}")
print(f"P(F ∨ G) = {disjunction_norm:.3f}")
print(f"P(¬F) = {negation_norm:.3f}")
print(f"P(F → G) = {implication_norm:.3f}")

# Test Case 3: Simple probabilistic formulas (using BL-algebra directly)
print("\n--- Test Case 3: Direct BL-Algebra Calculations ---")

# Instead of problematic Atom usage, let's calculate BL-algebra operations directly
f_prob = 0.4  # P(F) = 0.4
g_prob = 0.6  # P(G) = 0.6

print(f"P(F) = {f_prob:.1f}")
print(f"P(G) = {g_prob:.1f}")

# Test BL-algebra operations manually
and_simple = f_prob * g_prob
or_simple = f_prob + g_prob - f_prob * g_prob
not_simple = 1.0 - f_prob
impl_simple = 1.0 - f_prob + f_prob * g_prob

print(f"P(F ∧ G) = F × G = {and_simple:.3f}")
print(f"P(F ∨ G) = F + G - F×G = {or_simple:.3f}")
print(f"P(¬F) = 1 - F = {not_simple:.1f}")
print(f"P(F → G) = 1 - F + F×G = {impl_simple:.3f}")

# Test Case 4: Multiple probability combinations
print("\n--- Test Case 4: Various Probability Combinations ---")

prob_values = [0.1, 0.3, 0.5, 0.7, 0.9]
print("P(F)\tP(G)\tF∧G\tF∨G\t¬F\tF→G")
print("-" * 50)

for pf in prob_values:
    for pg in prob_values:
        # Calculate BL-algebra operations
        and_val = pf * pg
        or_val = pf + pg - pf * pg
        not_val = 1.0 - pf
        impl_val = 1.0 - pf + pf * pg
        
        print(f"{pf:.1f}\t{pg:.1f}\t{and_val:.3f}\t{or_val:.3f}\t{not_val:.1f}\t{impl_val:.3f}")



=== BL-Algebra Properties Verification ===

--- Test Case 1: P(F) ≈ 0.3, P(G) ≈ 0.7 ---
P(F) = 0.305
P(G) = 0.704
P(F ∧ G) = P(F) × P(G) = 0.215
P(F ∨ G) = P(F) + P(G) - P(F)×P(G) = 0.794
P(¬F) = 1 - P(F) = 0.695
P(F → G) = 1 - P(F) + P(F)×P(G) = 0.910

--- Test Case 2: Normal Distribution Probabilities ---
P(F) = P(N(0,1) > 0) = 0.511
P(G) = P(N(0.5,1) > 0) = 0.700
P(F ∧ G) = 0.358
P(F ∨ G) = 0.853
P(¬F) = 0.489
P(F → G) = 0.847

--- Test Case 3: Direct BL-Algebra Calculations ---
P(F) = 0.4
P(G) = 0.6
P(F ∧ G) = F × G = 0.240
P(F ∨ G) = F + G - F×G = 0.760
P(¬F) = 1 - F = 0.6
P(F → G) = 1 - F + F×G = 0.840

--- Test Case 4: Various Probability Combinations ---
P(F)	P(G)	F∧G	F∨G	¬F	F→G
--------------------------------------------------
0.1	0.1	0.010	0.190	0.9	0.910
0.1	0.3	0.030	0.370	0.9	0.930
0.1	0.5	0.050	0.550	0.9	0.950
0.1	0.7	0.070	0.730	0.9	0.970
0.1	0.9	0.090	0.910	0.9	0.990
0.3	0.1	0.030	0.370	0.7	0.730
0.3	0.3	0.090	0.510	0.7	0.790
0.3	0.5	0.150	0.650	0.7	0.850
0.3	0.7	0.21

In [58]:
# LTN (Logic Tensor Networks) Semantics Examples
print("=== LTN Semantics: p-norm Quantifiers vs Probabilistic Semantics ===")

# Create test domain and formula that shows differences
domain = [0.2, 0.7, 0.8, 0.9]
formula = var_greater("x", 0.5)

print(f"Domain: {domain}")
print(f"Formula: x > 0.5")
print(f"Formula values: {[formula.evaluate(ProgramState({'x': val})) for val in domain]}")

print("\n=== Universal Quantification Comparison ===")

# Standard probabilistic (inf-based)
prob_forall = forall("x", domain, formula)
prob_result = prob_forall.evaluate(ProgramState({}))
print(f"Probabilistic ∀x: {prob_result:.3f} (minimum)")

# LTN with different p values
for p in [1.0, 2.0, 5.0, float('inf')]:
    ltn_forall_q = ltn_forall("x", domain, formula, p)
    result = ltn_forall_q.evaluate(ProgramState({}))
    p_name = "∞" if p == float('inf') else str(p)
    print(f"LTN ∀x (p={p_name}): {result:.3f}")

print("\n=== Existential Quantification Comparison ===")

# Standard probabilistic (sup-based) 
prob_exists = exists("x", domain, formula)
prob_result = prob_exists.evaluate(ProgramState({}))
print(f"Probabilistic ∃x: {prob_result:.3f} (maximum)")

# LTN with different p values
for p in [1.0, 2.0, 5.0, float('inf')]:
    ltn_exists_q = ltn_exists("x", domain, formula, p)
    result = ltn_exists_q.evaluate(ProgramState({}))
    p_name = "∞" if p == float('inf') else str(p)
    print(f"LTN ∃x (p={p_name}): {result:.3f}")

# Test with mixed domain (some true, some false)
print("\n=== Mixed Domain Test: Key Differences ===")
mixed_domain = [0.3, 0.7, 0.8, 0.9]  # 0.3 > 0.5 is False
mixed_formula = var_greater("x", 0.5)

print(f"Mixed domain: {mixed_domain}")
print(f"Formula values: {[mixed_formula.evaluate(ProgramState({'x': val})) for val in mixed_domain]}")

# Compare quantifications
prob_forall_mixed = forall("x", mixed_domain, mixed_formula).evaluate(ProgramState({}))
ltn_forall_mixed = ltn_forall("x", mixed_domain, mixed_formula, p=2.0).evaluate(ProgramState({}))

prob_exists_mixed = exists("x", mixed_domain, mixed_formula).evaluate(ProgramState({}))
ltn_exists_mixed = ltn_exists("x", mixed_domain, mixed_formula, p=2.0).evaluate(ProgramState({}))

print(f"Probabilistic ∀x: {prob_forall_mixed:.3f} vs LTN ∀x (p=2): {ltn_forall_mixed:.3f}")
print(f"Probabilistic ∃x: {prob_exists_mixed:.3f} vs LTN ∃x (p=2): {ltn_exists_mixed:.3f}")

print("\n=== Hyperparameter Sensitivity Analysis ===")

# Test different p values on the mixed domain  
print("Universal Quantification with different p values:")
for p in [1.0, 1.5, 2.0, 3.0, 5.0, 10.0, float('inf')]:
    ltn_result = ltn_forall("x", mixed_domain, mixed_formula, p).evaluate(ProgramState({}))
    p_name = "∞" if p == float('inf') else f"{p:.1f}"
    print(f"  p={p_name}: {ltn_result:.3f}")

print("\nExistential Quantification with different p values:")
for p in [1.0, 1.5, 2.0, 3.0, 5.0, 10.0, float('inf')]:
    ltn_result = ltn_exists("x", mixed_domain, mixed_formula, p).evaluate(ProgramState({}))
    p_name = "∞" if p == float('inf') else f"{p:.1f}"
    print(f"  p={p_name}: {ltn_result:.3f}")

print("\n=== Convenience Functions ===")

# Test convenience functions
strict_forall = ltn_forall_strict("x", mixed_domain, mixed_formula).evaluate(ProgramState({}))
mean_forall = ltn_forall_mean("x", mixed_domain, mixed_formula).evaluate(ProgramState({}))
strict_exists = ltn_exists_strict("x", mixed_domain, mixed_formula).evaluate(ProgramState({}))
mean_exists = ltn_exists_mean("x", mixed_domain, mixed_formula).evaluate(ProgramState({}))

print(f"Strict ∀x (p→∞): {strict_forall:.3f}")
print(f"Mean ∀x (p=1): {mean_forall:.3f}")
print(f"Strict ∃x (p→∞): {strict_exists:.3f}")
print(f"Mean ∃x (p=1): {mean_exists:.3f}")

print("\n=== Intermediate Probability Values Example ===")

# Create a custom formula that returns the actual probability value (not boolean)
class ProbabilityFormula(ProbabilisticFormula):
    """Formula that returns the variable value itself as probability."""
    def __init__(self, variable: str):
        self.variable = variable
    
    def evaluate(self, state: ProgramState) -> float:
        return state.get(self.variable, 0.0)

# Test with intermediate probability values (not just 0/1)
probability_domain = [0.1, 0.4, 0.6, 0.9]
probability_formula = ProbabilityFormula("x")  # Returns x itself as probability

print(f"Discrete domain with intermediate probabilities: {probability_domain}")
print(f"Formula values (x itself): {[probability_formula.evaluate(ProgramState({'x': val})) for val in probability_domain]}")

# Compare quantifications with intermediate probability values
prob_forall_prob = forall("x", probability_domain, probability_formula).evaluate(ProgramState({}))
ltn_forall_prob = ltn_forall("x", probability_domain, probability_formula, p=2.0).evaluate(ProgramState({}))

prob_exists_prob = exists("x", probability_domain, probability_formula).evaluate(ProgramState({}))
ltn_exists_prob = ltn_exists("x", probability_domain, probability_formula, p=2.0).evaluate(ProgramState({}))

print(f"Probabilistic ∀x: {prob_forall_prob:.3f} vs LTN ∀x (p=2): {ltn_forall_prob:.3f}")
print(f"Probabilistic ∃x: {prob_exists_prob:.3f} vs LTN ∃x (p=2): {ltn_exists_prob:.3f}")

# Show detailed p-mean calculation for transparency
values = [0.1, 0.4, 0.6, 0.9]
manual_p2_mean = (sum(v**2 for v in values) / len(values))**(1/2)
print(f"Manual p=2 mean calculation: ({sum(v**2 for v in values)} / {len(values)})^(1/2) = {manual_p2_mean:.3f}")

print("\n=== All other operations (∧,∨,¬,→) remain the same as probabilistic semantics ===")


=== LTN Semantics: p-norm Quantifiers vs Probabilistic Semantics ===
Domain: [0.2, 0.7, 0.8, 0.9]
Formula: x > 0.5
Formula values: [0.0, 1.0, 1.0, 1.0]

=== Universal Quantification Comparison ===
Probabilistic ∀x: 0.000 (minimum)
LTN ∀x (p=1.0): 0.750
LTN ∀x (p=2.0): 0.866
LTN ∀x (p=5.0): 0.944
LTN ∀x (p=∞): 0.000

=== Existential Quantification Comparison ===
Probabilistic ∃x: 1.000 (maximum)
LTN ∃x (p=1.0): 0.750
LTN ∃x (p=2.0): 0.500
LTN ∃x (p=5.0): 0.242
LTN ∃x (p=∞): 1.000

=== Mixed Domain Test: Key Differences ===
Mixed domain: [0.3, 0.7, 0.8, 0.9]
Formula values: [0.0, 1.0, 1.0, 1.0]
Probabilistic ∀x: 0.000 vs LTN ∀x (p=2): 0.866
Probabilistic ∃x: 1.000 vs LTN ∃x (p=2): 0.500

=== Hyperparameter Sensitivity Analysis ===
Universal Quantification with different p values:
  p=1.0: 0.750
  p=1.5: 0.825
  p=2.0: 0.866
  p=3.0: 0.909
  p=5.0: 0.944
  p=10.0: 0.972
  p=∞: 0.000

Existential Quantification with different p values:
  p=1.0: 0.750
  p=1.5: 0.603
  p=2.0: 0.500
  p=3.0: 